In [ ]:
import keras
import pandas as pd
import numpy as np
import tqdm
import matplotlib.pyplot as plt
import os
from leaky_unit import ProportionalAddition
from PIL import Image

from build_prop_model import build_model_prop as bmp
from build_prop_model_backwards import build_model_prop as bmpb

from build_prop_model import build_model_prop_3t as bmp3t
from build_prop_model_backwards import build_model_prop_3t as bmpb3t

In [ ]:
def load_images(paths):
    images = []
    age = []
    gender = []
    ethnicity = []
    for path in paths:
        img = (np.asarray(Image.open(path)))[:,:,:3]
        path_split = path.split('/')[-1].split('_')
        age.append(path_split[0][-2:])
        gender.append(path_split[1])
        ethnicity.append(path_split[2][0])
        images.append(img[:,:,:3]/255)
    
    return np.array(images), [np.array(age), np.array(gender), np.array(ethnicity)]

In [ ]:
def compute_scores(model, tasks, image_paths, model_name):
    
    metrics = model.metrics_names
    
    metric_tasks = {metric:i for i, metric in enumerate(metrics)}
    
    
    
    age_loss = []
    gender_acc = []
    eth_acc = []
    
    dicc_tasks = {"age": age_loss, "gender": gender_acc, "ethnicity":eth_acc
    }
    
    dicc_tasks2 = {"age": "age_loss", "gender": "gender_acc", "ethnicity":"ethnicity_acc"
    }
    
    for i in range(0, len(image_paths)):
        
        images,  labels = load_images(image_paths[i*256:(i+1)*256])
        
        if len(images.shape) != 4: break
            
        if len(tasks) == 3:
            
            
            new_labels = [0,0,0]
            new_labels[metric_tasks['age_loss']-1] = labels[0]
            new_labels[metric_tasks['gender_loss']-1] = labels[1]
            new_labels[metric_tasks['ethnicity_loss']-1] = labels[2]

            scores = model.evaluate(images, new_labels)
            dicc_tasks[tasks[0]].append(scores[metric_tasks[dicc_tasks2[tasks[0]]]])
            dicc_tasks[tasks[1]].append(scores[metric_tasks[dicc_tasks2[tasks[1]]]])
            dicc_tasks[tasks[2]].append(scores[metric_tasks[dicc_tasks2[tasks[2]]]])
        
        if len(tasks) == 2:
            
            
            if tasks == ['age', 'gender']:
                new_labels = [0,0]
                new_labels[metric_tasks['age_loss']-1] = labels[0]
                new_labels[metric_tasks['gender_loss']-1] = labels[1]
                
                scores = model.evaluate(images, new_labels)
                age_loss.append(scores[metric_tasks['age_loss']])
                gender_acc.append(scores[metric_tasks['gender_acc']])
            
            if tasks == ['age', 'ethnicity']:
                new_labels = [0,0]
                new_labels[metric_tasks['age_loss']-1] = labels[0]
                new_labels[metric_tasks['ethnicity_loss']-1] = labels[2]
                
                scores = model.evaluate(images, new_labels)
                age_loss.append(scores[metric_tasks['age_loss']])
                eth_acc.append(scores[metric_tasks['ethnicity_acc']])
            
            if tasks == ['gender', 'ethnicity']:
                new_labels = [0,0]
                new_labels[metric_tasks['gender_loss']-1] = labels[1]
                new_labels[metric_tasks['ethnicity_loss']-1] = labels[2]
                
                scores = model.evaluate(images, new_labels)
                gender_acc.append(scores[metric_tasks['gender_acc']])
                eth_acc.append(scores[metric_tasks['ethnicity_acc']])
        
    
    return [model_name, division_by_zero(sum(age_loss),len(age_loss)), 
            division_by_zero(sum(gender_acc),len(gender_acc)), 
            division_by_zero(sum(eth_acc),len(eth_acc))]


In [ ]:
def division_by_zero(a,b):
    try:
        return a/b
    except:
        return '-'

In [ ]:
image_paths = os.listdir('DyadicFacesSimple/')
real_paths = []
for path in image_paths:
    real_paths.append('DyadicFacesSimple/'+path)

In [ ]:
df = []

model_path = 'Models/prop1_ageethnicityb.h5'
model = bmpb(nb_prop_layers=1, tasks=['age', 'ethnicity'])
model.load_weights(model_path)
df.append(compute_scores(model, ["age",  "ethnicity"], real_paths, "Baseline Model"))

In [ ]:
df

In [22]:
# baseline
df = []

model_path = 'FirstTake/baseline_agegenderethnicity.h5'
model = keras.models.load_model(model_path)
df.append(compute_scores(model, ["age", "gender", "ethnicity"], real_paths, "Baseline Model"))

#Hard Param norm 2t

for tasks in [["age", "gender"], ["age", "ethnicity"], ["gender", "ethnicity"]]:
    
    for shared_layers in range(1,6):
        
        model_path = 'Models/hardparam{}{}{}.h5'.format(shared_layers, tasks[0], tasks[1])
        model = keras.models.load_model(model_path)
        df.append(compute_scores(model, tasks, real_paths, "HPS {} shared layers {} {}".format(
            shared_layers, tasks[0], tasks[1])))
        
#Hard Param norm 3t
print("Finished")

for tasks in [["age", "gender", "ethnicity"]]:
    
    for shared_layers in range(1,6):
        
        model_path = 'Models/hardparam{}{}{}{}.h5'.format(shared_layers, tasks[0], tasks[1], tasks[2])
        model = keras.models.load_model(model_path)
        df.append(compute_scores(model, tasks, real_paths, "HPS {} shared layers 3 tasks".format(
            shared_layers)))
        
#Hard Param back 2t

for tasks in [["age", "gender"], ["age", "ethnicity"], ["gender", "ethnicity"]]:
    
    for shared_layers in range(1,6):
        
        model_path = 'Models/hardparam{}{}{}b.h5'.format(shared_layers, tasks[0], tasks[1])
        model = keras.models.load_model(model_path)
        df.append(compute_scores(model, tasks, real_paths, "HPS {} shared layers {} {} Backwards".format(
            shared_layers, tasks[0], tasks[1])))
        
#Hard Param back 3t

for tasks in [["age", "gender", "ethnicity"]]:
    
    for shared_layers in range(1,6):
        
        model_path = 'Models/hardparam{}{}{}{}b.h5'.format(shared_layers, tasks[0], tasks[1], tasks[2])
        model = keras.models.load_model(model_path)
        df.append(compute_scores(model, tasks, real_paths, "HPS {} shared layers 3 tasks Backwards".format(
            shared_layers)))
        
        
        
#Prop norm 2t

for tasks in [["age", "gender"], ["age", "ethnicity"], ["gender", "ethnicity"]]:
    
    for shared_layers in range(1,6):
        
        model_path = 'Models/prop{}_{}{}.h5'.format(shared_layers, tasks[0], tasks[1])
        model = bmp(tasks, shared_layers)
        model.load_weights(model_path)
        df.append(compute_scores(model, tasks, real_paths, "CS Network {} CS units  {} {}".format(
            shared_layers, tasks[0], tasks[1])))
        
#Prop norm 3t

for tasks in [["age", "gender", "ethnicity"]]:
    
    for shared_layers in range(1,6):
        
        model_path = 'Models/prop{}_{}{}{}.h5'.format(shared_layers, tasks[0], tasks[1], tasks[2])
        model = bmp3t(tasks, shared_layers)
        model.load_weights(model_path)
        df.append(compute_scores(model, tasks, real_paths, "CS Network {} CS units  3 tasks".format(
            shared_layers)))
        
#Prop back 2t

for tasks in [["age", "gender"], ["age", "ethnicity"], ["gender", "ethnicity"]]:
    
    for shared_layers in range(1,6):
        
        model_path = 'Models/prop{}_{}{}b.h5'.format(shared_layers, tasks[0], tasks[1])
        model = bmpb(tasks, shared_layers)
        model.load_weights(model_path)
        df.append(compute_scores(model, tasks, real_paths, "CS Network {} CS units  {} {} Backwards".format(
            shared_layers, tasks[0], tasks[1])))
        
#Prop 3t

for tasks in [["age", "gender", "ethnicity"]]:
    
    for shared_layers in range(1,6):
        
        model_path = 'Models/prop{}_{}{}{}b.h5'.format(shared_layers, tasks[0], tasks[1], tasks[2])
        model = bmpb3t(tasks, shared_layers)
        model.load_weights(model_path)
        df.append(compute_scores(model, tasks, real_paths, "CS Network {} CS units 3 tasks Backwards".format(
            shared_layers)))
        

235/235 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 4s 17ms/step
Finished
235/235 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 7s 31ms/step


256/256 [==============================] - ETA: 1: - ETA: 42s - ETA: 25 - ETA: 16 - ETA: 10 - ETA: 6 - ETA:  - 20s 79ms/step


235/235 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 11s 48ms/step


In [29]:
final_df = pd.DataFrame(df, columns=['Model', 'Age', 'Gender', 'Ethnicity'])

In [24]:
final_df

,Model,Age,Gender,Ethnicity
0,Baseline Model,268.435,0.67923,0.561295
1,HPS 1 shared layers age gender,305.834,0.731749,-
2,HPS 2 shared layers age gender,246.918,0.598903,-
3,HPS 3 shared layers age gender,234.871,0.641282,-
4,HPS 4 shared layers age gender,489.928,0.573371,-
5,HPS 5 shared layers age gender,284.552,0.45482,-
6,HPS 1 shared layers age ethnicity,235.982,-,0.479712
7,HPS 2 shared layers age ethnicity,299.023,-,0.535588
8,HPS 3 shared layers age ethnicity,406.232,-,0.517138
9,HPS 4 shared layers age ethnicity,524.272,-,0.370229


In [30]:
final_df = final_df.replace('-', 999)
final_df = final_df.round(3)
final_df = final_df.replace(999, '-')

In [31]:
final_df

,Model,Age,Gender,Ethnicity
0,Baseline Model,268.435,0.679,0.561
1,HPS 1 shared layers age gender,305.834,0.732,-
2,HPS 2 shared layers age gender,246.918,0.599,-
3,HPS 3 shared layers age gender,234.871,0.641,-
4,HPS 4 shared layers age gender,489.928,0.573,-
5,HPS 5 shared layers age gender,284.552,0.455,-
6,HPS 1 shared layers age ethnicity,235.982,-,0.48
7,HPS 2 shared layers age ethnicity,299.023,-,0.536
8,HPS 3 shared layers age ethnicity,406.232,-,0.517
9,HPS 4 shared layers age ethnicity,524.272,-,0.37


In [32]:
final_df.to_csv('ResultsMTLAndreuDyadicSimpleall.csv', index=False)

In [15]:
final_df = pd.read_csv('ResultsMTLAndreuDyadicSimpleall.csv').replace(0.0, )

,Model,Age,Gender,Ethnicity
0,Baseline Model,268.43543374792057,0.6792303856382979,0.5612948803191489
1,HPS 1 shared layers age gender,1868.9030362636486,0.4297872340425532,-
2,HPS 2 shared layers age gender,1878.659021905128,0.3148936170212766,-
3,HPS 3 shared layers age gender,1861.8600790876023,0.4317403590425532,-
4,HPS 4 shared layers age gender,1867.062668625852,0.5702127659574469,-
5,HPS 5 shared layers age gender,1865.3840583476615,0.4297872340425532,-
6,HPS 1 shared layers age ethnicity,1865.5392160050412,-,0.0
7,HPS 2 shared layers age ethnicity,1868.1696497491066,-,0.5068484042553192
8,HPS 3 shared layers age ethnicity,1861.8754455079425,-,0.0
9,HPS 4 shared layers age ethnicity,1864.1393678462252,-,0.19361702127659575


# Visualization

In [ ]:
model_path = 'FirstTake/baseline_agegenderethnicity.h5'
model = keras.models.load_model(model_path)

In [ ]:
np.asarray(Image.open(real_paths[a]))

In [ ]:
import random
from PIL import Image
from IPython.display import clear_output

while len(os.listdir('DyadicFacesSimple2/')) < 100:
    
    a = random.randint(0, len(real_paths)-1)

    img, labels = load_images([real_paths[a]])
    if 'DyadicFacesSimple2/'+real_paths[a].split('/')[-1] in os.listdir('DyadicFacesSimple2/'):
        continue
    plt.imshow(plt.imread(real_paths[a]))
    plt.show()
    print(model.predict((np.asarray(Image.open(real_paths[a]))/255)[:,:,:3].reshape(1,200,200,3))), labels

    b = input('Good Image')

    if b == 'y':
        plt.imsave('DyadicFacesSimple2/'+real_paths[a].split('/')[-1], np.asarray(Image.open(real_paths[a]))[:,:,:3])

    clear_output()

In [ ]:
plt.imshow(img.reshape(200,200,3)*255)

In [ ]:
img